# Neural network models

This notebook picks up after the `simple_models` notebook. After trying a range of classification algorithms, we'll try out some of the neural network models in [1]. These include fully-connected models of varying layer sizes, and finally convolutional models including the famous LeNet-5. 

Along the way, we'll be using Keras which is a library sitting on top of Theano or Tensorflow. This allows easy construction, training and evaluation of neural nets. Before we get started, here's a recap of the `simple_models` notebook models.

`[1]` - [Gradient-Based Learning Applied to Document Recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf),  LeCun et al, Nov 1998

In [1]:
# Load in dependencies, may not use all of these
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

%matplotlib inline

## Load pickle files

The original data files are processed using the `convert_data.py` script, and written out to pickle files. We can load these in as numpy arrays.

In [2]:
# Set up the file directory and names
DIR = '../input/'
X_TRAIN = DIR + 'train-images-idx3-ubyte.pkl'
Y_TRAIN = DIR + 'train-labels-idx1-ubyte.pkl'
X_TEST = DIR + 't10k-images-idx3-ubyte.pkl'
Y_TEST = DIR + 't10k-labels-idx1-ubyte.pkl'

def load_data():
    '''Loads pickled ubyte files with MNIST data
    INPUT: X_train_file, y_train_file - strings with training filenames
           X_test_file, y_test_File - strings with test filenames
    RETURNS: Tuple with (X_train, y_train, X_test, y_test)
    '''
    print('Loading pickle files ...')
    try:
        X_train = pickle.load( open( X_TRAIN, "rb" ) )
        y_train = pickle.load( open( Y_TRAIN, "rb" ) )
        X_test = pickle.load( open( X_TEST, "rb" ) )
        y_test = pickle.load( open( Y_TEST, "rb" ) )
    except:
        print('Error loading pickle file')
        return None
        
    return (X_train, y_train, X_test, y_test)

X_train, y_train, X_test,  y_test = load_data()

Loading pickle files ...


## Helper functions

Before evaluating some models on the images, let's create some helper functions we can re-use later on. These deal with converting images to and from 1d and 2d versions, plotting images, resizing them, etc.

In [3]:
def flatten_images(X):
    ''' Converts images to 1-d vectors
    INPUT: X - Input array of shape [n, w, h]
    RETURNS: Numpy array of shape [n, w*h]
    '''
    n, w, h = X.shape
    X_flat = X.reshape((n, w * h))
    return X_flat

def square_images(X, w, h):
    '''Converts single-vector images into square images 
    INPUT: X - numpy array of images in single-vector form
           w - width of images to convert to
           h - height of images to convert to
    RETURNS: Numpy array of shape [n, w, h]
    '''
    assert X.shape[1] == w * h, "Error - Can't square array of shape {} to {}".format(X.shape, (w, h))
    n = X.shape[0]
    X_square = X.reshape((n, w, h))
    return X_square


N_TRAIN, W, H = X_train.shape
N_TEST, w_test, h_test = X_test.shape

# Flatten the images
X_train = flatten_images(X_train)
X_test = flatten_images(X_test)

# Do some checks on the data
assert N_TRAIN == 60000, 'Error - expected 60000 training images, got {}'.format(N_TRAIN)
assert N_TEST == 10000, 'Error - expected 60000 training images, got {}'.format(N_TEST)
assert W == w_test, 'Error - width mismatch. Train {}, Test {}'.format(w, w_test)
assert H == h_test, 'Error - height mismatch. Train {}, Test {}'.format(h, h_test)

assert np.array_equal(X_train, flatten_images(square_images(X_train, W, H)))
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

print('Loaded train images shape {}, labels shape {}'.format(X_train.shape, y_train.shape))
print('Loaded test images shape {}, labels shape {}'.format(X_test.shape, y_test.shape))


Loaded train images shape (60000, 784), labels shape (60000, 1)
Loaded test images shape (10000, 784), labels shape (10000, 1)


## Data preparation

This section sets up global constants used in all models (to ensure a fair comparison). It also prepares the data by converting y values to one-hot, and normalizing X inputs.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler

# Keras Common configuration
SEED = 1234 # Fix the seed for repeatability
N_JOBS=-2 # Leave 1 core free for UI updates
VERBOSE=2 # 3 is the most verbose level
NB_EPOCH = 50 # todo ! Check how many epochs in the paper
BATCH = 256 # todo ! Check this in the paper too


def stratified_subsample(X, y, num_rows):
    '''Creates a stratified subsample of X and y
    INPUT: X and y, numpy arrays
    RETURNS: subset of X and y, maintaining class balances
    '''
    # Create a stratified, shuffled subset of the training data if needed
    N = X.shape[0]
    if num_rows < N:
        print('Reducing size from {} to {} examples'.format(N, num_rows))
        new_X, _, new_y, _ = train_test_split(X_train, y_train, 
                                              train_size=N, random_state=SEED)    

def onehot_encode_y(y_train, y_test):
    '''Convert y_train and y_test to a one-hot encoding version
    INPUT: y_train - np.array of size (n_train,)
           y_test - np.array of size (n_test,)
    RETURNS: y_train - np.array of size (n_train, n_classes)
             y_test - np.arary of size (n_test, n_classes)
    '''    
    print('Converting y variables to one-hot encoding..')
    lbe = LabelBinarizer()
    lbe.fit(y_train)
    y_train = lbe.transform(y_train)
    y_test = lbe.transform(y_test)
    return y_train, y_test

def z_norm_X(X_train, X_test):
    '''Z-normalizes X_train and X_test with 0 mean and 1 std. dev.
    INPUT: X_train - training set
           X_test - test set
    RETURNS: X_train - normalized version of same size
             X_test - normalized version (using X_train parameters)
    '''
    print('Z-normalizing X data..')    
    std = StandardScaler()
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    std.fit(X_train)
    X_train = std.transform(X_train)
    X_test = std.transform(X_test)
    return X_train, X_test
    
y_train, y_test = onehot_encode_y(y_train, y_test)
X_train, X_test = z_norm_X(X_train, X_test)

print('Train images shape {}, labels shape {}'.format(X_train.shape, y_train.shape))
print('Test images shape {}, labels shape {}'.format(X_test.shape, y_test.shape))

Converting y variables to one-hot encoding..
Z-normalizing X data..
Train images shape (60000, 784), labels shape (60000, 10)
Test images shape (10000, 784), labels shape (10000, 10)


## [1] C.5 - One hidden layer models

The paper continues with an evaluation of single hidden-layer models. We'll be training a selection of three models from the paper, whhich are listed below.  All use sigmoid activations.

* Model a - 28x28-300-10: 4.7% Error
* Model b - 20x20-300-10:  1.6% Error (images were reduced to 20x20 and centred in 28x28  background)
* Model c - 28x28-1000-10: 4.5% Error


### Model a

This is a baseline FC network which should give 4.7% error. 

In [5]:
# Model a - 28x28-300-10: 4.7% Error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(300, input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))
print('Model summary:\n')
model.summary()

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print('\nTraining model\n')
model.fit(X_train, y_train,
          nb_epoch=NB_EPOCH,
          batch_size=BATCH,
          verbose=2)

print('\nGenerating predictions on test set\n')
scores = model.evaluate(X_test, y_test, batch_size=BATCH)
errors = [1.0 - score for score in scores]

print('\n\nTest set training error {:.4f}, test error {:.4f}'.format(errors[0], errors[1]))

Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5105)
/home/tim/anaconda3/envs/tensorflow/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Model summary:

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 300)           235500      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 300)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 10)            3010        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 10)            0           dense_2[0][0]                    
Total params: 238510
______________________________________________________

### Model b 

This model requires some preprocessing of the input data, to shrink the images into a 20x20 area centred in a 28x28 image. After that we need to remember to Z-Normalize them again.

In [ ]:
X_train_input.shape

In [ ]:
# Check a few training values at random as a sanity check
def show_label_images(X, y, images=None):
    '''Shows random images in a grid
    INPUT: X - image data
           y - class label
           images - indexes of images to show. Randomly selected if None
    RETURNS: Nothing.
    '''
    
    num = 4
    num_square = num ** 2
    
    if images is None:
        images = np.random.randint(0, n_train, num_square)
        
    print('Showing training image indexes {}'.format(images))

    fig, axes = plt.subplots(num,num, figsize=(6,6))
    for idx, val in enumerate(images):
        r, c = divmod(idx, num)
        ax = axes[r][c]
        ax.imshow(X[images[idx]], cmap=plt.cm.binary)
        ax.annotate('Label: {}'.format(y[val]), xy=(1, 1))
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

show_label_images(X_train_input, y_train_input)

In [ ]:
from PIL import Image
from tqdm import *

def resize_image(image, img_size, back_size, back_color):
    '''Resizes the image to `new_size`, pastes in center of backgrond
    INPUT: image = Input image
           img_size = tuple of images new dimensions
           back_size = tuple of new image background dimensions
           back_color = color to fill background with
    RETURNS: New image
    '''

    new_img = Image.new('L', back_size, (back_color,))
    img = Image.fromarray(image)
    img.thumbnail(img_size, Image.ANTIALIAS) 

    h_diff = int((back_size[0] - img_size[1]) / 2)
    w_diff = int((back_size[1] - img_size[1]) / 2)

    new_img.paste(img, (h_diff, w_diff))
    return np.asarray(new_img)

def resize_image_array(X, img_size, back_size, back_color, desc=None):
    '''Resizes images held in a numpy array'''
    X_out = np.zeros_like(X)
    n_images = X.shape[0]
    for idx in tqdm(range(n_images), desc=desc): # "Resizing {} images".format(n_images)):
        X_out[idx] = resize_image(X[idx], img_size, back_size, 0)
        
    return X_out

X_train_resize = resize_image_array(X_train_input, (20,20), (28,28), 0, 'Resizing training images')
X_test_resize = resize_image_array(X_test_input, (20,20), (28,28), 0, 'Resizing training images')

# Reshape the images so they're a single row in the numpy array
X_train_resize = X_train_resize_float.reshape((n_train, w * h))
X_test_resize = X_test_resize_float.reshape((n_test, w * h))


print('Z-normalizing X data')
std = StandardScaler()
X_train_resize_float = X_train_resize.astype(np.float32)
X_test_resize_float = X_test_resize.astype(np.float32)



std.fit(X_train_resize_float.astype(np.float32))
X_train = std.transform(X_train_resize_float)
X_test = std.transform(X_test_resize_float)





In [ ]:
print(w, h)

In [ ]:
# Model a - 28x28-300-10: 4.7% Error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(300, input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))
print('Model summary:\n')
model.summary()

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print('\nTraining model\n')
model.fit(X_train, y_train,
          nb_epoch=NB_EPOCH,
          batch_size=BATCH,
          verbose=2)

print('\nGenerating predictions on test set\n')
scores = model.evaluate(X_test, y_test, batch_size=BATCH)
errors = [1.0 - score for score in scores]

print('\n\nTest set training error {:.4f}, test error {:.4f}'.format(errors[0], errors[1]))

In [ ]:
model.summary()

## 